In [2]:
import os, sys
from tabulate import tabulate
from z3 import set_param
from debugger.trace_analyzer import shorten_qid
from debugger3 import Debugger3
from utils.system_utils import log_info
from tqdm import tqdm
import pandas as pd

cwd = os.getcwd()
os.chdir("/home/yizhou7/mariposa")

UNSTABLE_10_SECS = {
    "data/projs/flowcert/base.z3/permission.14.smt2",
    "data/projs/splinterdb/base.z3/allocation_layer__LikesBetree_v__LikesBetree.3.smt2",
    "data/projs/splinterdb/base.z3/allocation_layer__LikesBetree_v.18.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_eventually_always_no_delete_resource_request_msg_in_flight.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__validation.3.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__stateful_set_match__lemma_from_after_get_stateful_set_step_to_after_update_stateful_set_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof.2.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__validation.3.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__zookeeper_api__lemma_zk_request_implies_step_helper.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_update_resource_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__liveness__stateful_set_match.2.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match.6.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__proof__lemma_eventually_always_every_resource_create_request_implies_at_after_create_resource_step.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__liveness__daemon_set_match__lemma_from_after_get_daemon_set_step_to_after_update_daemon_set_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_update_resource_step.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_create_resource_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__resource_match.1.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__proof__lemma_eventually_always_object_in_response_at_after_create_resource_step_is_same_as_etcd.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__stateful_set_match__lemma_stateful_set_state_matches_at_after_update_stateful_set_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__validation__lemma_always_stateful_set_in_create_request_msg_satisfies_unchangeable.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_always_no_update_status_request_msg_in_flight_of_except_stateful_set.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match.5.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__proof.9.smt2",
    "data/projs/verismo.dedup/base.z3/boot__init__e820_init.2.smt2",
    "data/projs/verismo.dedup/base.z3/security__monitor.21.smt2",
    "data/projs/verismo.dedup/base.z3/vbox__vbox.5.smt2",
    "data/projs/verismo.dedup/base.z3/arch__ptram__ptram_p2.smt2",
    "data/projs/verismo.dedup/base.z3/bsp__ap.3.smt2",
    "data/projs/verismo.dedup/base.z3/snp__cpuid.17.smt2",
    "data/projs/verismo.dedup/base.z3/snp__ghcb__proto_page.141.smt2",
    "data/projs/verismo.dedup/base.z3/pgtable_e__pte.20.smt2",
    "data/projs/verismo.dedup/base.z3/vbox__vbox.4.smt2",
    "data/projs/verismo.dedup/base.z3/boot__init__e820_init_alloc.1.smt2",
    "data/projs/verismo.dedup/base.z3/allocator__linkedlist.10.smt2",
    "data/projs/verismo.dedup/base.z3/pgtable_e__pte.18.smt2",
    "data/projs/verismo.dedup/base.z3/boot__init__e820_init.4.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.13.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__span_queue_delete.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__setup_v.5.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-commit_segment.1.smt2",
    "data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.3.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.17.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.12.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.16.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-config.35.smt2",
    "data/projs/vsystemsnew/base.z3/noderep-smt-spec__cyclicbuffer.5.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.2.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__logimpl_v.6.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.5.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__start_v.2.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.14.smt2",
    "data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.4.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg.69.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.17.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__setup_v.6.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.11.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_os_alloc.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.7.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.4.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg__impl_%4__merge_with_before_ll_inv_valid_unused.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_span_free_coalesce_before.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.15.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.9.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.2.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.13.smt2",
    "data/projs/vsystemsnew/base.z3/noderep-smt-spec__cyclicbuffer.3.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.10.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.14.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__layout_v.30.smt2",
    "data/projs/atmosphere/base.z3/kernel__create_and_share_pages.5.smt2",
}


In [3]:
def load_scores():
    scores = dict()

    for line in open("/home/yizhou7/mariposa/doc/scores.csv", "r"):
        parts = line.strip().split(",")
        query = parts[0]
        scores[query] = [int(s) for s in parts[1:]]
        
    return scores


In [4]:
scores = load_scores()

print("total unstable:", len(UNSTABLE_10_SECS))
print("no proofs:", len(UNSTABLE_10_SECS - set(scores.keys())))

no_fixes = set()
bad_scores = set()

for query, score in scores.items():
    if score[0] == 0:
        no_fixes.add(query)
    elif all([s == 0 for s in score[1:]]):
        bad_scores.add(query)

print("no fixes:", len(no_fixes))
print("bad scores:", len(bad_scores))

total unstable: 70
no proofs: 4
no fixes: 14
bad scores: 26


In [5]:
for query in sorted(bad_scores):
    print(query, scores[query][0])

data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__proof.9.smt2 1
data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__validation.3.smt2 3
data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_create_resource_step.smt2 30
data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_always_no_update_status_request_msg_in_flight_of_except_stateful_set.smt2 41
data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__zookeeper_api__lemma_zk_request_implies_step_helper.smt2 35
data/projs/atmosphere/base.z3/kernel__create_and_share_pages.5.smt2 5
data/projs/verismo.dedup/base.z3/pgtable_e__pte.18.smt2 1
data/projs/vsystemsnew/base.z3/mimalloc-smt-commit_segment.1.smt2 1
data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_os_alloc.smt2 10
data

In [ ]:
query = "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.11.smt2"
dbg = Debugger3(query)
dbg.evaluate_rankings()

[INFO] [init] analyzing data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.11.smt2 in dbg/b689a8d455 
[WARN] proof QI graph is cyclic! 
parsing trace: dbg/b689a8d455/traces/reseed.14883523172079684349
[INFO] [graph] found dbg/b689a8d455/graphs/reseed.14883523172079684349.txt 
[WARN] [report] already exists: dbg/b689a8d455/report.csv 
./src/analysis_wizard.py stable -e default -s z3_4_13_0 -i data/projs/singleton_b689a8d455.filtered/base.z3
[INFO] [edit] 145 edits registered 
[INFO] found 2 stabilizing edits (excluding prelude_fuel_defaults) 
qid             eid       action         time    v0    v1    v2    v3    v4    v5
--------------  --------  -----------  ------  ----  ----  ----  ----  ----  ----
prelude_sub     31984b41  instantiate    0.2     10    32    32    70    68    70
prelude_eucmod  3d6bf970  instantiate    0.22   129   127   153   131   186   186


[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [7]:
df = pd.read_csv("dbg/92c260e39d/report.csv", float_precision='round_trip').round(0)
df["qid"] = df["qid"].apply(shorten_qid)
print(tabulate(df.sort_values(by='trace count', ascending=False), headers='keys', tablefmt='psql'))

+-----+-------------------------------------------------------------------------------------+-------------+---------------+---------------+------+------+------+-------+-------+-------+
|     | qid                                                                                 | action      |   trace count |   proof count |   v0 |   v1 |   v2 |    v3 |    v4 |    v5 |
|-----+-------------------------------------------------------------------------------------+-------------+---------------+---------------+------+------+------+-------+-------+-------|
|   0 | all                                                                                 | -           |         20506 |           749 |  nan |  nan |  nan |   nan |   nan |   nan |
| 122 | prelude_unbox_box_int                                                               | instantiate |          1241 |            29 |  460 |  623 | 1148 |  1224 |  1181 |  1181 |
| 140 | internal_vstd!seq.Seq.index.?_pre_post_definition                  

In [8]:
query = "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__validation.3.smt2"
dbg = Debugger3(query)
dbg.evaluate_rankings()

[INFO] [init] analyzing data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__validation.3.smt2 in dbg/a1b4b3fd4a 
[INFO] proof QI graph is acyclic 
parsing trace: dbg/a1b4b3fd4a/traces/rename.17017982759078069243
[INFO] [graph] found dbg/a1b4b3fd4a/graphs/rename.17017982759078069243.txt 
[WARN] [report] already exists: dbg/a1b4b3fd4a/report.csv 
./src/analysis_wizard.py stable -e default -s z3_4_13_0 -i data/projs/singleton_a1b4b3fd4a.filtered/base.z3
[WARN] skip due to encountered: [ERROR] there are 50 queries with files, but no experiments done  
./src/analysis_wizard.py stable -e verus_quick -s z3_4_13_0 -i data/projs/singleton_a1b4b3fd4a.filtered/base.z3
[INFO] [edit] 788 edits registered 
[INFO] found 3 stabilizing edits (excluding prelude_fuel_defaults) 
qid                                                                                  eid       action         time    v0    v1    v2    v3    v4    v5
---------------------------------------

[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
df = pd.read_csv("dbg/a1b4b3fd4a/report.csv", float_precision='round_trip').round(0)
ranks = df["v0"].rank(ascending=False)
df["qid"] = df["qid"].apply(shorten_qid)
print(tabulate(df.sort_values(by='trace count', ascending=False), headers='keys', tablefmt='psql'))

+-----+-------------------------------------------------------------------------------------+-------------+---------------+---------------+-------+--------+-------+------------------+------------------+------------------+
|     | qid                                                                                 | action      |   trace count |   proof count |    v0 |     v1 |    v2 |               v3 |               v4 |               v5 |
|-----+-------------------------------------------------------------------------------------+-------------+---------------+---------------+-------+--------+-------+------------------+------------------+------------------|
|   0 | all                                                                                 | -           |        394261 |          1134 |   nan |    nan |   nan |    nan           |    nan           |    nan           |
| 382 | prelude_unbox_box_bool                                                              | error       |     